In [ ]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import kagglehub

# Download latest version
# path = kagglehub.dataset_download("allanwandia/sepsis")
df = pd.read_csv("/kaggle/input/sepsis/Paitients_Files_Train - Paitients_Files_Train.csv")  # Adjust filename if needed

# Remove units from numerical columns
def remove_units(value):
    return float(re.findall(r"[\d.]+", str(value))[0]) if re.search(r"[\d.]+", str(value)) else np.nan

columns_with_units = ["SK", "TS", "PR", "PL", "M11", "BD2"]  # List of columns to clean
for col in columns_with_units:
    df[col] = df[col].apply(remove_units)

# Convert the target column to binary (0 = Negative, 1 = Positive)
df["Sepssis"] = df["Sepssis"].map({"Negative": 0, "Positive": 1})
df= df.drop(columns=["ID", "Insurance"])
# Handle missing values (impute with mean)
df.fillna(df.mean(), inplace=True)

# Split dataset into features (X) and target (y)
X = df.drop(columns=["Sepssis"])
y = df["Sepssis"]

# Normalize the data

print(X)


In [7]:
import keras

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Build the ANN model
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 98ms/step - accuracy: 0.5137 - loss: 0.6803 - val_accuracy: 0.7167 - val_loss: 0.6005
Epoch 2/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7378 - loss: 0.5672 - val_accuracy: 0.7000 - val_loss: 0.5600
Epoch 3/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7831 - loss: 0.4878 - val_accuracy: 0.7167 - val_loss: 0.5658
Epoch 4/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7675 - loss: 0.4732 - val_accuracy: 0.7667 - val_loss: 0.5729
Epoch 5/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8069 - loss: 0.4187 - val_accuracy: 0.7583 - val_loss: 0.5789
Epoch 6/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7965 - loss: 0.4281 - val_accuracy: 0.7583 - val_loss: 0.5642
Epoch 7/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8011 - loss: 0.4292 - val_accuracy: 0.7667 - val_loss: 0.5629
Epoch 8/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7950 - loss: 0.4208 - val_accuracy: 0.7333 - val_loss: 0.5801
Ep

In [5]:
# Train an XGBoost Decision Tree Model
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV

# Define XGBoost model
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic', 
    eval_metric='logloss',
    use_label_encoder=False,
    tree_method='gpu_hist'
)

# Define parameter grid
param_grid = {
    'max_depth': [3, 5, 7],              # Tree depth
    'learning_rate': [0.01, 0.1],   # Step size
    'n_estimators': [100, 200],     # Number of trees
    'subsample': [0.7, 0.8, 1.0],        # Fraction of samples per tree
    'colsample_bytree': [0.7, 0.8, 1.0], # Feature sampling
    'gamma': [0.1, 0.2]               # Regularization
}

# Grid Search with 5-Fold Cross-Validation
grid_search = GridSearchCV(
    estimator=xgb_model, 
    param_grid=param_grid, 
    scoring='accuracy', 
    cv=5, 
    verbose=1, 
    n_jobs=-1
)

# Fit model
grid_search.fit(X_train, y_train)

# Best parameters and best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("Best Hyperparameters:", best_params)

# Make predictions
y_pred = best_model.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Tuned XGBoost Test Accuracy: {accuracy:.4f}")

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [16:02:16] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Best Hyperparameters: {'colsample_bytree': 0.8, 'gamma': 0.2, 'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.7}
Tuned XGBoost Test Accuracy: 0.7167


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [16:02:16] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
